<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/02_Supervised_Modelisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [1]:
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
#import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
#from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
random_state = 747

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **2.Data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X_questions = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/X_tfidf_500.csv')
y_tags = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/y_onehot_500.csv')

In [5]:
X_questions = X_questions.drop(['Unnamed: 0'], axis=1).iloc[:, 1:]
X_questions.head()

,static,function,struct,mongodb,object,api,date,laravel,post,text,input,kera,module,dataframe,panda,c,list,random,filter,class,html,java,tensorflow,numpy,seaborn,matplotlib,python,logging,csv,validation,iterator,lambda,graph,plot,collection,generator,regex,nginx,php,linux,...,mocking,excel,plotly,shiny,woocommerce,apollo,jasmine,blazor,tuples,xaml,generic,tsql,hive,ssis,vectorization,inheritance,swagger,tkinter,hibernate,winforms,elixir,cypress,hadoop,opengl,nlp,mockito,uwp,lstm,msbuild,keycloak,delphi,ansible,socket,gatsby,oop,cocoapods,winapi,raku,yarnpkg,jestjs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.137624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.344969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
y_tags = y_tags.drop(['Unnamed: 0'], axis=1)
y_tags.columns = y_tags.columns.str.replace("Tag_st_", "")
y_tags.head()

,.net,.net-core,aggregation-framework,airflow,ajax,algorithm,amazon-cloudformation,amazon-cognito,amazon-dynamodb,amazon-ec2,amazon-s3,amazon-web-services,anaconda,android,android-architecture-components,android-constraintlayout,android-fragments,android-gradle-plugin,android-jetpack,android-layout,android-livedata,android-recyclerview,android-room,android-studio,androidx,angular,angular-cli,angular-material,angular-material2,angular5,angular6,angular7,angularjs,animation,annotations,ansible,apache,apache-kafka,apache-spark,apache-spark-sql,...,variables,vb.net,vba,vector,vectorization,video,vim,visual-c++,visual-studio,visual-studio-2017,visual-studio-2019,visual-studio-code,vscode-settings,vue-component,vue.js,vuejs2,vuetify.js,vuex,web,web-scraping,webpack,websocket,webview,winapi,windows,windows-10,winforms,woocommerce,wordpress,wpf,x86,xamarin,xamarin.android,xamarin.forms,xaml,xcode,xcode11,xml,yaml,yarnpkg
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **3.Modelisation**

In [7]:
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

### **Train Test Split**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_questions, y_tags,
                                                    test_size=0.3, 
                                                    random_state=random_state)

In [9]:
X_train.head()

,static,function,struct,mongodb,object,api,date,laravel,post,text,input,kera,module,dataframe,panda,c,list,random,filter,class,html,java,tensorflow,numpy,seaborn,matplotlib,python,logging,csv,validation,iterator,lambda,graph,plot,collection,generator,regex,nginx,php,linux,...,mocking,excel,plotly,shiny,woocommerce,apollo,jasmine,blazor,tuples,xaml,generic,tsql,hive,ssis,vectorization,inheritance,swagger,tkinter,hibernate,winforms,elixir,cypress,hadoop,opengl,nlp,mockito,uwp,lstm,msbuild,keycloak,delphi,ansible,socket,gatsby,oop,cocoapods,winapi,raku,yarnpkg,jestjs
15114,0.0,0.0,0.0,0.0,0.234308,0.400446,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206208,0.291883,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.711095,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27789,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9638,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316420,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22907,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.441367,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20646,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.078072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.047569,0.0,0.908086,0.0,0.0,0.057032,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.268781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
y_train.head()

,.net,.net-core,aggregation-framework,airflow,ajax,algorithm,amazon-cloudformation,amazon-cognito,amazon-dynamodb,amazon-ec2,amazon-s3,amazon-web-services,anaconda,android,android-architecture-components,android-constraintlayout,android-fragments,android-gradle-plugin,android-jetpack,android-layout,android-livedata,android-recyclerview,android-room,android-studio,androidx,angular,angular-cli,angular-material,angular-material2,angular5,angular6,angular7,angularjs,animation,annotations,ansible,apache,apache-kafka,apache-spark,apache-spark-sql,...,variables,vb.net,vba,vector,vectorization,video,vim,visual-c++,visual-studio,visual-studio-2017,visual-studio-2019,visual-studio-code,vscode-settings,vue-component,vue.js,vuejs2,vuetify.js,vuex,web,web-scraping,webpack,websocket,webview,winapi,windows,windows-10,winforms,woocommerce,wordpress,wpf,x86,xamarin,xamarin.android,xamarin.forms,xaml,xcode,xcode11,xml,yaml,yarnpkg
15114,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27789,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20646,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### **OneVsRestClass and GridSearchCV**

### **Logistic Regression**

In [11]:
lr_model = OneVsRestClassifier(LogisticRegression())
lr_hyperparameters = dict(estimator__C=[10**-2, 10**0, 10**2])
lr_cv = GridSearchCV(lr_model, lr_hyperparameters, cv=5, n_jobs = -1, scoring= 'accuracy')

In [12]:
lr_cv.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=5, error_score=nan,
             estimator=OneVsRestClassifier(estimator=LogisticRegression(C=1.0,
                                                                        class_weight=None,
                                                                        dual=False,
                                                                        fit_intercept=True,
                                                                        intercept_scaling=1,
                                                                        l1_ratio=None,
                                                                        max_iter=100,
                                                                        multi_class='auto',
                                                                        n_jobs=None,
                                                                        penalty='l2',
                                                                        random_state=None

In [13]:
lr_cv.best_params_

{'estimator__C': 100}

In [14]:
lr_cv.score(X_test, y_test)

0.17216666666666666

### **Random Forest**

In [ ]:
#dt_classifier = OneVsRestClassifier(DecisionTreeClassifier())
#dt_hyperparameters =  dict(estimator__min_samples_split=[10, 20, 40], estimato__min_samples_leaf=[0.05, 0.1, 0.2], estimator__criterion=['gini', 'entropy']) 
#dt_cv = GridSearchCV(OneVsRestClassifier(DecisionTreeClassifier()), dt_hyperparameters, cv=5, n_jobs = -1, scoring= 'accuracy')

In [ ]:
#dt_model = DecisionTreeClassifier()
#dt_hyperparameters = 
#dt_cv = GridSearch(dt_model, dt_hyperparameters, X_train, y_train)

In [16]:
dt_model = Pipeline([('dt', OneVsRestClassifier(estimator = DecisionTreeClassifier()))])
dt_hyperparameters = {'dt__estimator__min_samples_split':[10, 20, 40], 
                   'dt__estimator__min_samples_leaf': [0.05, 0.1, 0.2], 
                   'dt__estimator__criterion':['gini', 'entropy']}

dt_cv = GridSearchCV(dt_model, dt_hyperparameters, scoring='accuracy', cv=5, n_jobs = -1)                   

In [18]:
sorted(dt_model.get_params().keys())

['dt',
 'dt__estimator',
 'dt__estimator__ccp_alpha',
 'dt__estimator__class_weight',
 'dt__estimator__criterion',
 'dt__estimator__max_depth',
 'dt__estimator__max_features',
 'dt__estimator__max_leaf_nodes',
 'dt__estimator__min_impurity_decrease',
 'dt__estimator__min_impurity_split',
 'dt__estimator__min_samples_leaf',
 'dt__estimator__min_samples_split',
 'dt__estimator__min_weight_fraction_leaf',
 'dt__estimator__presort',
 'dt__estimator__random_state',
 'dt__estimator__splitter',
 'dt__n_jobs',
 'memory',
 'steps',
 'verbose']

In [19]:
dt_cv.fit(X_train, y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('dt',
                                        OneVsRestClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                             class_weight=None,
                                                                                             criterion='gini',
                                                                                             max_depth=None,
                                                                                             max_features=None,
                                                                                             max_leaf_nodes=None,
                                                                                             min_impurity_decrease=0.0,
                                                                                       

In [20]:
dt_cv.best_params_

{'dt__estimator__criterion': 'entropy',
 'dt__estimator__min_samples_leaf': 0.05,
 'dt__estimator__min_samples_split': 10}

In [21]:
dt_cv.score(X_test, y_test)

0.07316666666666667

### **Naive Bayes**

In [22]:
nb_model = Pipeline([('nb', OneVsRestClassifier(estimator = MultinomialNB()))])
nb_hyperparameters = {'nb__estimator__alpha': [0.01, 0.1, 1]}   

nb_cv = GridSearchCV(nb_model, nb_hyperparameters, scoring='accuracy', cv=5, n_jobs = -1)      

In [23]:
sorted(nb_model.get_params().keys())

['memory',
 'nb',
 'nb__estimator',
 'nb__estimator__alpha',
 'nb__estimator__class_prior',
 'nb__estimator__fit_prior',
 'nb__n_jobs',
 'steps',
 'verbose']

In [24]:
nb_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('nb',
                                        OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0,
                                                                                    class_prior=None,
                                                                                    fit_prior=True),
                                                            n_jobs=None))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'nb__estimator__alpha': [0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [25]:
nb_cv.score(X_test, y_test)

0.13208333333333333

In [26]:
nb_cv.best_params_

{'nb__estimator__alpha': 0.01}